### Загрузите данные и подготовьте их к анализу

Загрузите данные о визитах, заказах и рекламных расходах из CSV-файлов в переменные.

**Пути к файлам**

- визиты: `/datasets/visits_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/visits_info_short.csv);
- заказы: `/datasets/orders_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/orders_info_short.csv);
- расходы: `/datasets/costs_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/costs_info_short.csv).

Изучите данные и выполните предобработку. Есть ли в данных пропуски и дубликаты? Убедитесь, что типы данных во всех колонках соответствуют сохранённым в них значениям. Обратите внимание на столбцы с датой и временем.

In [2]:
# для начала импортируем библиотеки 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt 

ModuleNotFoundError: No module named 'pandas'

Загрузим данные из файлов 

In [ ]:
visits = pd.read_csv('/datasets/visits_info_short.csv')
orders = pd.read_csv('/datasets/orders_info_short.csv')
costs = pd.read_csv('/datasets/costs_info_short.csv')
# выведем по 5 строк, чтобы посмотреть на данные
print('visits')
display(visits.head(5))
print('orders')
display(orders.head(5))
print('costs')
display(costs.head(5))

Приведём названия столбцов к привычному snake_case - сейчас в названиях есть заглавные буквы и пробелы, это неудобно. 

In [ ]:
visits.rename(columns = {'User Id' : 'user_id','Region' : 'region','Device' : 'device','Channel' : 'channel'
                         ,'Session Start' : 'session_start','Session End' : 'session_end'}, inplace = True )
display(visits.head(5))
orders.rename(columns = {'User Id':'user_id', 'Event Dt' : 'event_dt', 'Revenue' : 'revenue'}, inplace = True)
display(orders.head(5))
costs.rename(columns = {'Channel' : 'channel'}, inplace = True)
display(costs.head(5))

Изучим типы данных и посмотрим, где требуется преобразование. 

In [ ]:
visits.info()
orders.info()
costs.info()

In [ ]:
# приведём данные по датам к datetime сразу для всех трёх таблиц
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
costs['dt'] = pd.to_datetime(costs['dt'])
# и сразу выведем информацию о таблицах 
visits.info()
orders.info()
costs.info()

Поменяли тип данных для столбцов, касающихся даты. Для остальных столбцов, похоже, необходимости менять тип данных нет. 
Проверим наличие явных дубликатов

In [ ]:
print("Явных дубликатов в visits" , visits.duplicated().sum())
print("Явных дубликатов в orders" , orders.duplicated().sum())
print("Явных дубликатов в costs" , costs.duplicated().sum())

Явных дубликатов не нашли, теперь посмотрим на пропуски. 

In [ ]:
print('Пропуски в visits')
display(visits.isna().sum())
print('Пропуски в orders')
display(orders.isna().sum())
print('Пропуски в costs')
display(costs.isna().sum())

Пропусков в данных нет ни в одном столбце. Видимо, данные выгрузили из системы, где контролируется заполнение нужных нам полей. 

**Общий вывод по шагу 1**: загружены данные для анализа, выполнена предобработка: приведены к общему виду названия столбцов, приведены к удобному для расчётов типу datetime столбцы, касающиеся дат. Данные изучены на наличие пропусков и явных дубликатов - пропусков и явных дубликатов не обнаружено. Данные можно брать в дальнейшую работу. 

### Задайте функции для расчёта и анализа LTV, ROI, удержания и конверсии.

Разрешается использовать функции, с которыми вы познакомились в теоретических уроках.

Это функции для вычисления значений метрик:

- `get_profiles()` — для создания профилей пользователей,
- `get_retention()` — для подсчёта Retention Rate,
- `get_conversion()` — для подсчёта конверсии,
- `get_ltv()` — для подсчёта LTV.

А также функции для построения графиков:

- `filter_data()` — для сглаживания данных,
- `plot_retention()` — для построения графика Retention Rate,
- `plot_conversion()` — для построения графика конверсии,
- `plot_ltv_roi` — для визуализации LTV и ROI.

Объявим вышеупомянутые функции для вычисления значений метрик. 

In [ ]:
# функция для создания пользовательских профилей

def get_profiles(sessions, orders, ad_costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['dt'] = pd.to_datetime(profiles['dt'], format="%Y-%m-%d")
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # добавляем флаги для всех событий из event_names
    '''for event in event_names:
        if event in events['event_name'].unique():
            profiles[event] = profiles['user_id'].isin(
                events.query('event_name == @event')['user_id'].unique()
            )'''

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles


In [ ]:
# функция для расчёта удержания
def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
# функция для расчёта конверсии
def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
# функция для расчёта LTV и ROI

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

Сразу также объявим функции для визуализации метрик 

In [ ]:
# функция для сглаживания фрейма

def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df

In [ ]:
# функция для визуализации удержания

def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show()

In [ ]:
# функция для визуализации конверсии

def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

In [ ]:
# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 10))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

**Вывод по шагу 2:** задали необходимые для дальнейшего анализа функции. Можно приступать к следующему шагу

<div class="alert alert-success">
<b>👍 Успех:</b> Все функции готовы!
</div>

### Исследовательский анализ данных

- Составьте профили пользователей. Определите минимальную и максимальную даты привлечения пользователей.
- Выясните, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих из каждой страны.
- Узнайте, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого устройства.
- Изучите рекламные источники привлечения и определите каналы, из которых пришло больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

После каждого пункта сформулируйте выводы.

Составим профили пользователей и определим минимальную и максимальную дату привлечения пользователей. 


In [ ]:
# получаем профили пользователей
profiles = get_profiles(visits, orders, costs)
print(profiles.head(5)) 

Получили профили, теперь высчитываем даты. Время нам не требуется, будем считать по полю dt

In [ ]:
min_analysis_date = profiles['dt'].min()
observation_date = profiles['dt'].max() 
print('Минимальная дата привлечения пользователей', min_analysis_date)
print('Максимальная дата привлечения пользователей', observation_date)

**Вывод по шагу исследования**: Составили профили пользователей и посчитали минимальную и максимальную даты привлечения пользователя. 
Минимальная дата привлечения пользователей - 2019-05-01
Максимальная дата привлечения пользователей - 2019-10-27 

Теперь выясним,  
- из каких стран пользователи приходят в приложение 
- и на какую страну приходится больше всего платящих пользователей. 

Построим таблицу, отражающую 
- количество пользователей 
- и долю платящих из каждой страны.

Для начала посмотрим, сколько вообще пользователей у нас есть из каких стран, и сколько платящих пользователей есть в стране.

In [ ]:
display(profiles
            .groupby(['region'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

display(profiles.query('payer == True')
            .groupby(['region'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

Больше всего пользователей в Штатах - 100002 против "второго места", UK, где количество пользователей - 17575. Платящих пользователей больше всего тоже в Штатах - что логично для этой количественной метрики, ведь в Штатах общее количество пользователей больше. Однако, это не говорит ничего о доле платящих пользователей, эту метрику нужно подсчитать. Построим таблицу с количеством и долей платящих пользователей. Для этого можно просто взять aggfunction mean, поскольку в поле payer будет значение 1, если пользователь платящий, и 0 - если нет. Соответственно среднее как раз и покажет нам долю платящих пользователей, можно сделать нужную таблицу одной строкой. 

In [ ]:
profiles.groupby('region').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}
                              ).sort_values(by=('payer', 'mean'), ascending=False)



Получается, что и доля платящих пользователей выше в US. Интересные результаты показывает Германия - при относительно небольшом количестве пользователей доля платящих пользователей - 4.11%, что выше, чем у UK и Франции, в которых общее число пользователей выше. 
Однако, в целом разброс числа пользователей у этих стран не так высок, поэтому какие-либо выводы делать не стоит. 

**Вывод по шагу**: наибольшее число пользователей, наибольшее число платящих пользователей и самая высокая доля платящих пользователей - в Штатах. Европейские страны имеют более низкие показатели - как по числу пользователей, так и по числу платящих пользователей. 


Далее изучим, какими устройствами пользуются клиенту, и какие устройства в фаворе у платящихпользователей. Построим таблицу с количеством пользователей и долей платящих для каждого устройства. 

Как и на прошлом шаге, посмотрим сначала количество пользователей по каждому девайсу в принципе и количество платящих пользователей с их устройствами

In [ ]:
display(profiles
            .groupby(['device'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

display(profiles.query('payer == True')
            .groupby(['device'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

Больше всего пользователей - владельцы iphone, для платящих пользователей ситуация в целом почти такая же, только Mac немного опережает PC. Изучим долю платящих пользователей и сразу отсортируем данные по ней. Воспользуемся принципом подсчёта с предыдущего шага

In [ ]:
profiles.groupby('device').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}
                              ).sort_values(by=('payer', 'mean'), ascending=False)


Самую высокую конверсию показывает Mac - 6.36 процента. На втором месте iphone, далее Андроид и ПК закрывает рейтинг с самым низким результатом - почти 5.05 процента. Пользователи ПК предпочитают прокрастинировать бесплатно. 

**Вывод по шагу**: больше всего пользователей - владельцы iphone, та же ситуация с платящими пользователеями, количественно среди платящих пользователей больше всего владельцев phone. А вот с конверсией ситуация несколько другая: самый большой процент платящих пользователей - владельцы Mac.




Теперь изучим источники привлечения, нам потребуется 
- определить каналы, из которых пришло больше всего платящих пользователей
- построить таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения 

Как и на предыдущих шагах, построим таблицы по каналам и затем построим сводную таблицу, используя aggfunction

In [ ]:
display(profiles
            .groupby(['channel'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

display(profiles.query('payer == True')
            .groupby(['channel'])
            .agg({'user_id':'count'})
            .sort_values(['user_id'], ascending=False)
            .reset_index())

Больше всего пользователей - это канал organic, "органические" пользователи, которые пришли сами и которые для нас считаются "бесплатными" (но среди них тоже есть платящие пользователи), второе и третье место со значительным отрывом от первого занимают FaceBoom и TipTop. Количество же платящих пользователей наибольшее у FaceBoom, за ним идёт TipTop, и на третьем месте расположился канал  organic. 

In [ ]:
profiles.groupby('channel').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}
                              ).sort_values(by=('payer', 'mean'), ascending=False)

**Вывод по шагу**: количественно больше всего платящих пользователей пришло по каналам FaceBoom, TipTop, organic. Доля платящих пользователей же самая высокая у FaceBoom - 12.2 процента из 29144 пользователей, а вот 2 и 3 места занимают каналы AdNoSense (11.34 процента из 3880 пользователей) и lambdaMediaAds (10.47 процентов из 2149 пользователей). TipTop с небольшим отставанием (9.6 процента) держится на четвёртом месте, а вот канал organic - лидер по количеству пользователей в принципе - держится на последнем месте с результатом в 2 процента конверсии. В 6 раз меньше, чем у лидера, FaceBoom.

**Общий вывод по этапу**. Провели исследовательский анализ данных - собрали профили пользователей, изучили количество пользователей и конверсию в платежи в разрезе по регионам, по устройствам и по каналам привлечения. Заметили следующее: 
* по регионам 
Наибольшее количество пользователей, платящих пользователей и наибольшая доля платящих пользователей - в US 
* по устройствам 
Наибольшее количество пользователей и платящих пользователей - владельцы iPhone, а вот наибольшая доля платящих пользователей - среди пользователей Mac. Впрочем, Iphone по данному показателю не сильно отстаёт, речь о десятых долях процентов. 
* по каналам привлечения
Количественно больше всего пользователей - органические. Платящих пользователей больше всего пришло с канала FaceBoom, наибольшая доля платящих пользователей тоже у данного канала. TipTop тоже неплохо себя показывает, доля платящих пользователе1 9.6 процента при относительно большом количестве пользователей. 

### Маркетинг 
- Посчитайте общую сумму расходов на маркетинг.
- Выясните, как траты распределены по рекламным источникам, то есть сколько денег потратили на каждый источник.
- Постройте график с визуализацией динамики изменения расходов во времени по неделям по каждому источнику. Затем на другом графике визуализируйте динамику изменения расходов во времени по месяцам по каждому источнику.
- Узнайте, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника. Используйте профили пользователей.
Напишите промежуточные выводы.

In [ ]:
# выведем общую сумму всех расходов, посчитав её по таблице costs
display(costs['costs'].sum())

In [ ]:
# кажется, что 11 знак после запятой нетак важен, округлим значение и выведем округлённое 
print('Общая сумма расходов на маркетинг: ', round(costs['costs'].sum(), 1))

**Промежуточный вывод**: Общая сумма на расходы всех рекламных кампаний - 105497.3. Без оценочных суждений, поскольку окупаемость нам ещё предстоит подсчитать.

<div class="alert alert-success">
<b>👍 Успех:</b> Общая сумма расходов посчитана!
</div>

Изучим, как распределены расходы на рекламные кампании в разрезе по каналу. 

In [ ]:
costs.groupby('channel').agg({'costs': 'sum'}).sort_values(by='costs', ascending=False)

Хорошо показавшие себя в плане доли платящих пользователей сети - TipTop и FaceBoom - забрали на себя большую часть рекламного бюджета, половина всего бюджета ушла на TipTop, FaceBopm забрал треть, и остатки распределены по остальным сетям. 

**Вывод по шагу**: Больше всего потрачено на рекламную кампанию в TipTop (более половины бюджета), на втором месте FaceBoom (примерно треть бюджета), остальные сети делят между собой остаток бюджета. 

Постройте график с визуализацией динамики изменения расходов во времени по неделям по каждому источнику. Затем на другом графике визуализируйте динамику изменения расходов во времени по месяцам по каждому источнику.

Теперь надо построить график с визуализацией динамики изменения расходов во времени по неделям для каждого канала. 

Для работы с маркетинговыми кампаниями хотелось бы исключить из анализа органических пользователей - это по определению те пользователи, на привлечение которых не было рекламных кампаний, и соответственно расходов на них нет, и нет смысла обрабатывать лишние данные. На всякий случай проверим, не фигурирует ли данный канал (в связи с ошибкой в данных, неправильным пониманием данного канала и прочими возможными причинами) в данных о расходах


In [ ]:
costs.query('channel == "organic"').head()

Всё в порядке, можем исключить эти данные из анализа, чтобы снизить вычислительную нагрузку. 

In [ ]:
# фильтруем данные по каналу, чтобы построение отрабатывало быстрее 
profiles.query('channel != "organic"').pivot_table(
    index='dt', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Дата'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по датам');

In [ ]:
# проверяем, что график тот же без фильтрации 
profiles.pivot_table(
    index='dt', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Дата'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по датам');

In [ ]:
# строим по месяцам
profiles.query('channel != "organic"').pivot_table(
    index='month', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Месяц'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по месяцам');

В рамках предобработки не было добавлено поле week к профилям, сделаем это сейчас для построения графика по неделям 

In [ ]:
costs['week'] = costs['dt'].dt.isocalendar().week
profiles['week'] = profiles['dt'].dt.isocalendar().week

In [ ]:
# строим по неделям
profiles.query('channel != "organic"').pivot_table(
    index='week', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Неделя'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по неделям');

Посмотрим ещё на эти графики без учёта самых популярных сетей faceboom и tiptop. Исключим их из выборки, чтобы масштабировать основные сети



In [ ]:
# строим по неделям
profiles.query('channel != "organic" and channel != "FaceBoom" and channel != "TipTop"').pivot_table(
    index='week', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Неделя'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по неделям');

In [ ]:
# строим по месяцам
profiles.query('channel != "organic" and channel != "FaceBoom" and channel != "TipTop"').pivot_table(
    index='month', 
    columns='channel', 
    values='acquisition_cost',
    aggfunc='sum', 
).plot(figsize=(15, 5), grid=True)

plt.xlabel('Месяц'),
plt.ylabel('Расходы'),
plt.title('Динамика расходов по месяцам');

**Вывод по шагу**: Построили графики трат на рекламные кампании по неделям и месяцам. Для самых популярных сетей - FaceBoom b TipTop видим стабильный рост вложений в течение года, пик примерно в в сентябре-октябре. Интересно, что на TipTop в начале года вкладывались больше, чем в FaceBoom. 

Что касается менее "популярных" сетей, видим, что к июню сократились затраты на сеть AdNonSense (канал, который может похвастаться вторым местом по доле платящих пользователей), зато выросли затраты на WahooNetBanner. 



Далее нам требуется используя профили пользователей, узнать, сколько стоило привлечение одного пользователя из каждого источника. 



In [ ]:
# посчитаем среднее
user_cost_per_channel = (profiles
                        .query('channel != "organic"')
                        .groupby('channel')
                        .agg({'acquisition_cost':'mean'})
                        .sort_values('acquisition_cost', ascending=False)
                        .reset_index()
                        .rename(columns={'acquisition_cost': 'acquisition_cost_mean'})
                    )
user_cost_per_channel

In [ ]:
# на всякий случай также посчитаем и медиану 
user_cost_per_channel = (profiles
                        .query('channel != "organic"')
                        .groupby('channel')
                        .agg({'acquisition_cost':'median'})
                        .sort_values('acquisition_cost', ascending=False)
                        .reset_index()
                        .rename(columns={'acquisition_cost': 'acquisition_cost_median'})
                    )
user_cost_per_channel

In [ ]:
# выведем и среднее значение, и медиану
user_cost_per_channel = (profiles
                        .query('channel != "organic"')
                        .groupby('channel')
                        .agg(
                            acquisition_cost_mean=('acquisition_cost', 'mean'),
                            acquisition_cost_median=('acquisition_cost', 'median')
                        )
                        .sort_values('acquisition_cost_mean', ascending=False)
                        .reset_index()
                    )

user_cost_per_channel

В целом неважно, среднее мы берём или медиану, - топ одинаков - на первом месте расположился с большим отрывом TipTop, далее FaceBoom и AdNonSense. 
Нужно было найти среднюю стоимость, поэтому будем смотреть на среднее. 

**Вывод по шагу**: проанализировали стоимость привлечения пользователя по каждому каналу. Самые "дорогие" пользователи - из tiptop, второе и третье место - за сетями FaceBoom b AdNonSense. 

**Общий вывод по разделу**: Оценили общие затраты на рекламу - чуть более 100 тысяч; проанализировали затрату на рекламу по разным каналам, увидели, что половина бюджета потрачена на TipTop, примерно треть - на FaceBoom, остатки потрачены на все остальные сети. Самый "дорогой" пользователь - это пользователь с канала TipTop, второе и третье место по стоимости привлечения пользователя - у каналов FaceBoom и AdNonSense. 

Стоимость привлечения для канала TipTop, на который потрачена половина бюджета, значительно выше, чем для  остальных каналов. Самое дорогое привлечение соответственно - у канала TipTop - почти в 3 раза выше, чем у второго места. FaceBoom и AdNonSense, расположившиеся соответственно на втором и третьем местах, тоже значительно опережают остальные каналы, хотя здесь разница уже не так велика. Похоже, TipTop обходится очень дорого относительно других каналов, но нужно проверить и окупаемость рекламы. 

### Оцените окупаемость рекламы

Используя графики LTV, ROI и CAC, проанализируйте окупаемость рекламы. Считайте, что на календаре 1 ноября 2019 года, а в бизнес-плане заложено, что пользователи должны окупаться не позднее чем через две недели после привлечения. Необходимость включения в анализ органических пользователей определите самостоятельно.

- Проанализируйте окупаемость рекламы c помощью графиков LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проверьте конверсию пользователей и динамику её изменения. То же самое сделайте с удержанием пользователей. Постройте и изучите графики конверсии и удержания.
- Проанализируйте окупаемость рекламы с разбивкой по устройствам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по странам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по рекламным каналам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Ответьте на такие вопросы:
    - Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Чем могут быть вызваны проблемы окупаемости?

Напишите вывод, опишите возможные причины обнаруженных проблем и промежуточные рекомендации для рекламного отдела.

Начнём с первого пункта, укажем horizon_days в 2 недели, построим графики - нужные функции определены заранее. Как и ранее, органических пользователей учитывать не будем - они для нас бесплатные, на расчёты не влияют, а вот нагрузку на вычислительную систему мы сможем снизить, убрав эти данные из расчёта.

Для всех этапов будем использовать созданные ранее функции для подсчёта и визуализации данных
#### Общая окупаемость

In [ ]:
horizon_days = 14
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days
)

# строим графики
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days) 

**Промежуточные выводы по графикам**: 
- стоимость привлечения пользователей растёт;
- ROI меньше единицы на протяжении всего лайфтайма, т.е. прибыли по сути нет, реклама не окупается 
- реклама окупалась в мае, но щатем, начиная с июня, начинает работать в убыток 
Дополнительно заметим, что график LTV выглядит корректно, что подтверждает, что момент и горизонт анализа выбран верно

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>


Далее проверим конверсию - в разреще по каналам, регионам и устройствам
#### Конверсия 

##### Конверсия по регионам

In [ ]:
dimensions = ['region']
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

**Вывод по этапу**: В US значительно выше и конверсия, и динамика конверсии пользователей. Также можно отметить, что в мае очень высокая конверсия была во Франции и достаточно значительная - в Германии и UK, однако после июня показатели сильно падают

##### Конверсия по устройствам

In [ ]:
dimensions = ['device']
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

**Вывод по этапу**: Пользователи Mac имеют самую высокую конверсию в среднем. Худшую конверсию с большим отрывом показывает PC, динамика конверсии тоже даёт самые низкие показатели.

##### Конверсия по каналам


In [ ]:
dimensions = ['channel']
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

**Вывод по этапу**: самая высокая конверсия у FaceBoom, но ещё неплохо себя показывает AdNonSense. Такой дорогой с точки зрения вложений в рекламу TipTop тоже находится достаточно высоко в топе, на 4 месте. 


#### Удержание
Проанализируем удержание пользователей и динамику его изменения

##### Удержание в разрезе по каналам привлечения 

In [ ]:
dimensions = ['channel']
retention_raw, retention_grouped, retention_history = get_retention(
    profiles.query('channel != "organic"'), visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days)

**Вывод по этапу**: 
- стабильно ниже всех показатели удержания платящих пользователей у FaceBoom
- у AdNonSense тоже удержание не на высоте, предпоследнее место в топе 
- у TipTop ситуация с удержанием получше, наравне со всеми. Однако помним, что в TipTop вкладывают значительно больше денег, чем в остальные сети. 
- неплатящие пользователи с точки зрения удержания в целом практически одинаковы везде

##### Удержание в разрезе по регионам 

In [ ]:
dimensions = ['region']
retention_raw, retention_grouped, retention_history = get_retention(
    profiles.query('channel != "organic"'), visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days)

**Вывод по этапу**: 
- неплатящие пользователи примерно одинаковы в любых регионах, их графики удержания сильно похожи друг на друга 
- ситуация для платящих пользователей несколько отличается от ситуации с неплатящими - US показывает себя хуже всех, остальные примерно одинаковы. 
- зато динамика удержания у платящих пользователей гораздо более сглаженная, ситуация стабильная 

##### Удержание в разрезе по устройствам

In [ ]:
dimensions = ['device']
retention_raw, retention_grouped, retention_history = get_retention(
    profiles.query('channel != "organic"'), visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days)

**Вывод по этапу**: 
- ситуация по неплатящим пользователям в целом примерно одинакова для всех устройств 
- самое высокое удержание показывает PC - канал с худшей конверсией 
- в целом здесь графики устройств близки друг к другу, влияние канала на удержание не так уж велико 

Проанализируем окупаемость рекламы - с разбивкой по уже известным параметрам, - устройствам, регионам и каналам привлечения 
#### Окупаемость
##### Окупаемость в разрезе по устройствам

In [ ]:
dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=7
) 

Выводы по этапу: 
- LTV для разных устройств в целом близки друг к другу, тут выделяется только PC, где LTV заметно ниже
- зато стоимость привлечения пользователей ПК ниже, чем для остальных устройств - сюда вкладывались меньше всего 
- и не смотря на то, что в целом пользователи ПК приносят меньше денег (судя по LTV), но при этом это единственный канал, по которому выходим на окупаемость. 

Построим графики в разрезе по странам 

##### Окупаемость в разрезе по регионам

In [ ]:
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=7
) 

**Выводы по этапу**: 
- на окупаемость не выходит только регион US 
- при этом суммарно US приносит больше всех прибыли - это объяснимо, ведь в US в принципе количественно больше пользователей 
- интересно, что начиная с июня, затраты на рекламу существенно возросли в US и упали в остальных странах 
- к пятому дню все регионы уже выходят на окупаемость, кроме US 

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Посмотрим окупаемость в разрезе по каналам

##### Окупаемость в разрезе по каналам

In [ ]:
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles.query('channel != "organic"'), orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=21
) 

Потребовалось увеличить сглаживание в 3 раза, чтобы графики было удобнее смотреть. 


**Выводы по этапу**: 
- TipTop "съедает" почти весь рекламный бюджет, и при этом не окупается, в отличие от большинства каналов
- не окупаются каналы TipTop, FaceBoom и AdNonSense, при этом судя по динамике, AdNonSense иногда поднимается выше уровня окупаемости. FaceBoom и TipTop же стабильно ниже уровня окупаемости. 

Разберёмся с вопросами
- Окупается ли реклама, направленная на привлечение пользователей в целом?
- Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
- Чем могут быть вызваны проблемы окупаемости?

#### Окупаемость рекламы в целом 

Как уже отмечали при исследовании общей окупаемости, в целом реклама не окупается. График ROI растёт, но до уровня окупаемости не поднимается. 

#### Исследование влияния устройств, стран и каналов на окупаемость

- самые "дорогие" с точки зрения рекламы пользователи - владельцы Mac и Iphone. При этом на окупаемость они не выходят, окупается только канал ПК 
- при рассмотрении регионов видим, что значительно большие затраты идут на пользователей из US, при этом затраты не окупаются 
- с точки зрения каналов, самые дорогие из них - TipTop и FaceBoom. Но при этом TipTop, на который потрачено более половины рекламного бюджета, как уже отмечали в рамках исследования стоимостей, не окупается

Получается, основные траты идут на те направления, которые не окупаются, это менее заметно по устройствам, но влияние отчётливо видно для каналов и стран. TipTop - самый неокупаемый канал (хотя FaceBoom с его слабым удержанием тоже не очень хорош), US - самый неокупаемый регион.

#### Возможные причины проблем окупаемости 

Основная масса пользователей - это пользователи из US, пришедшие из TipTop, пользователи iphone. Получается, основная масса пользователей по сути привлечена убыточными рекламными кампаниями - теми, которые не окупаются, учитывая регион и канал. 
Возможно, вложения в данные убыточные категории того не стоят. 

**Общий вывод по разделу**: 
Проанализировали конверсию, удержание и окупаемость и сделали выводы по самым убыточным категориям. Разобрались с возможными причинами проблем окупаемости и в целом влиянием различных категорий на окупаемость. 

### Напишите выводы

- Выделите причины неэффективности привлечения пользователей.
- Сформулируйте рекомендации для отдела маркетинга.

**Общий вывод**
Загрузили и обработали данные для дальнейшего анализа. Далее провели исследование данных и получили следующие выводы: 

* US - лидеры по количеству пользователей, как платных, так и бесплатных. Страны Европы тут сильно отстают.
* Расходы на рекламу в US значительно выше остальных регионов, но они не окупаются. 
* В европейских регионах реклама окупается. При этом удержание пользователей в странах Европы выше, хотя конверсия значительно ниже, чем в US. 
* TipTop забрал более половины рекламного бюджета на привлечение пользователей, при этом TipTop не окупается. 
* Самую высокую конверсию показал FaceBoom, но удержание при этом для этой сети значительно ниже остальных. 
* Рекламные расходы в целом не окупаются. 

Рекомендации для отдела маркетинга: 
- обратить внимание на регионы Европы, пользователи в них оказываются лояльнее и в целом европейские регионы хорошо себя показывают в плане окупаемости рекламы. Возможно, стоит вложиться в это направление 
- обратить внимание на пользователей ПК: они показывают лучшее удержание, а вот с конверсией ситуация хуже всех. Реклама на них окупается, возможно, стоит вложиться в ПК пользователей. 
- стоит сократить расходы на рекламу в TipTop - потрачено на данный канал более половины бюджета, но показатели по нему не так хороши. Средняя конверсия, удержание не так плохо, но это единственный канал, который не окупается. 

Анализ вложений и окупаемости стоит повторить спустя время, чтобы проверить, как сработали рекомендации, как изменились тенденции и удалось ли вывести рекламу на окупаемость. 